# Copy Memory Task: 
### This task involves inputting a 1d array where the first value is the intended output, and is followed by zeros until a flag integer is seen, then the model should output the origin intended output value.
### Ex:
### In:   [6 0 0 0 0 0 0 0 0 0 9]
### Out:  [0 0 0 0 0 0 0 0 0 0 6]



In [1]:
from Training.generator_memory import generator
import numpy as np
from SSM.hippo import hippo_legs, discretize_bilinear
from SSM.helpers import sigmoid, binary_cross_entropy
from SSM.mmodel import SimpleSSM




 

In [2]:
#data parameters


#set data parameters
n_sequence = 1
n_train = 10000
n_test = 1000
time_step = 10
#n_batch

gen = generator(n_sequence, time_step, n_train)

x_train, y_train = gen.seq()

print(x_train[0:10])

gen = generator(n_sequence, time_step, n_test)

x_test, y_test = gen.seq()



[[5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [4. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [7. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [4. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]
 [6. 0. 0. 0. 0. 0. 0. 0. 0. 0. 9. 0.]]


In [3]:
xx = x_train[0].reshape(-1, 1)
xx.shape
yy = y_train[0].reshape(-1,1)
yy.shape

(12, 1)

In [4]:
#takes some time to run, if time_step = 10
model = SimpleSSM(input_dim = 1,
        output_dim = 10)
#print(y_train[0])
print(model.A_ct)
print(model.A)
prev = 1000000000
prev_a = 5
for j in range(100):
    
    epoch_loss = 0.0 
    for i in range(x_train.shape[0]):
        loss, grads = model.loss_and_grads_softmax(x_train[i].reshape(-1, 1), y_train[i].reshape(-1,1))
        model.step(grads, lr = 1e-5)
        epoch_loss += loss
    if epoch_loss/x_train.shape[0] > prev and prev_a != 5:
        prev_a = model.A.copy()
    if j % 10 == 0:
        if epoch_loss/x_train.shape[0] < prev:
            prev = epoch_loss/x_train.shape[0]
        else:
            break
            
    print('average loss= ' + str(epoch_loss/x_train.shape[0]))

print(prev_a)
print(model.A)



[[ -1.           0.           0.           0.           0.
    0.           0.           0.        ]
 [ -1.73205081  -2.           0.           0.           0.
    0.           0.           0.        ]
 [ -2.23606798  -3.87298335  -3.           0.           0.
    0.           0.           0.        ]
 [ -2.64575131  -4.58257569  -5.91607978  -4.           0.
    0.           0.           0.        ]
 [ -3.          -5.19615242  -6.70820393  -7.93725393  -5.
    0.           0.           0.        ]
 [ -3.31662479  -5.74456265  -7.41619849  -8.77496439  -9.94987437
   -6.           0.           0.        ]
 [ -3.60555128  -6.244998    -8.06225775  -9.53939201 -10.81665383
  -11.95826074  -7.           0.        ]
 [ -3.87298335  -6.70820393  -8.66025404 -10.24695077 -11.61895004
  -12.84523258 -13.96424004  -8.        ]]
[[ 3.33333333e-01 -3.37864825e-16 -2.06854932e-16  1.93156033e-16
   1.87791344e-16 -6.85528257e-17 -3.86372710e-16 -2.22044605e-16]
 [-5.77350269e-01  3.90787006e-16 

In [5]:
#once the loss starts to increase, it increaes indefintelyt

x_train[1], y_train[1], model.forward(x_train[1].reshape(-1, 1))

(array([4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 9., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 4.]),
 (array([[  10.22739575,    7.56605482,    8.01794499,    8.03635392,
             8.5383453 ,    8.43256802,    8.33805207,    8.15534345,
             7.27236088,    6.71109059],
         [  -9.46538475,  -13.04527781,  -12.40369966,  -12.74155952,
           -12.68035178,  -12.44606367,  -12.33214784,  -12.84473505,
           -13.25153517,  -13.39944007],
         [  23.38705272,   19.41363151,   19.4362648 ,   19.06598868,
            18.94995129,   18.98629371,   19.402986  ,   19.28641833,
            17.88070291,   18.20106906],
         [ -21.87771373,  -27.25433318,  -27.22759086,  -27.49081375,
           -27.94130604,  -27.81739987,  -27.2995638 ,  -28.29496654,
           -27.42925595,  -27.14895045],
         [  46.08912956,   40.82760101,   41.23746663,   40.02721049,
            39.77125028,   41.02172166,   40.30630668,   40.44139572,
            38.89333

In [6]:
for i in range(10):
    #print(x_train[i])
    #print(y_train[i])
    logits, _ = model.forward(x_train[i].reshape(-1,1))
    
    pred_digits = np.argmax(logits, axis=1)
    #print(pred_digits)
    if y_train[i].all() == pred_digits.all():
        print(pred_digits, y_train[i])

[0 0 0 0 0 0 0 0 0 0 0 7] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5.]
[0 0 0 0 0 0 0 0 0 0 0 7] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 4.]
[0 0 0 0 0 0 0 0 0 0 0 7] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 7.]
[0 0 0 0 0 0 0 0 0 0 0 0] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3.]
[0 0 0 0 0 0 0 0 0 0 0 7] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 4.]
[0 0 0 0 0 0 0 0 0 0 0 0] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 3.]
[0 0 0 0 0 0 0 0 0 0 0 0] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2.]
[0 0 0 0 0 0 0 0 0 0 0 0] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2.]
[0 0 0 0 0 0 0 0 0 0 0 0] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
[0 0 0 0 0 0 0 0 0 0 0 7] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 6.]


In [7]:
x_test.shape

(1000, 12)

In [8]:
N = x_test.shape[0]
total_loss = 0.0
total_correct = 0
total_bits = 0

for i in range(N):
    u = x_test[i].reshape(-1, 1)      # (T, 1)
    tgt = y_test[i].reshape(-1,1)    # (T, 1)

    # forward pass only (no grads)
    logits, _ = model.forward(u)

    # loss for this sequence
    pred_digits = np.argmax(logits, axis=1)


    # compute accuracy for this sequence
    #print(tgt.reshape(-1), pred_digits)
    if tgt.reshape(-1)[-1] == pred_digits[-1]:
        total_correct += 1
    
    


accuracy = total_correct / N

print(accuracy)


0.121
